# Neighborlists and Dangerous Builds

The construction of a neighbor list can drastically reduce the run time of a MD simulation. Rather than needing to compute distances (which are used as input to the forces) between all pairs of N atoms, when using a neighbor list, distances are computed only for those particles that are in a given particle's neighbor list. A neighbor list is constructed by identifying the particles that are within a specified distance of a given particle. The neighbor cutoff should be larger than the cutoff used in the force computation; this excess is typically referred to as the skin.

The neighborlist relies on the idea that over a shortwindow, the local environment around a particle doesn't change much. As a result, the costly operation of determining neighboring particles needs to be done infrequently, which speeds up computation.

The neighborlist has two basic parameters: (1) the skin size (i.e, buffer) and (2) the frequency of updating/checking for updates. These two parameters are coupled; for example, a system with high mobility may require a larger skin to avoid needing to be updated the neighborlist every timestep and to ensure particle interactions are not missed.

### The Skin
The skin is used to create a buffer of particles that are close by, but just outside of the interaction cutoff. This increases the number of calculations that must be done, since we still need to calculate the distance with all particles in the neighborlist even if a particle is ultimately be outside of the interaction range. However, by including this buffer of particles, neighborlists need to be updated less frequently, since we are basically keeping track of the location of nearby particles that we may interact with in the near future.



### Updating Frequency

In most codes, you have two ways of enforcing an update of the neighborlist.

For example, one approach is to specify that the neighborlist be updated every N steps. This approach tends to work well for systems that are dense or slowly moving, as particle motion should be relatively uniform in the system. However, this may make it challenging to balance wanting to reduce the number of updates that are done and ensuring that particles don't move too far, resulting in missed interactions (i.e., dangerous builds). This approach can speed things up, but should be considered "less safe".

Another approach is to check the displacement of particles. The general rule is that a neighborlist needs to be rebuilt if any particle has move more than skin/2.0, such to ensure that particle interactions are not being missed. This helps to ensure that "dangerous builds" aren't present in the system (See below). In many codes this also can be couple "checking" with a time modulation, e.g., only check every N timesteps.



### Dangerous Builds

Most codes will output a summary of the total number of dangerous builds, that is, the total number of times it built the neighborlist was rebuilt with particles move greater than skin/2.0. In such cases, interactions may be missed, resulting in misleading and incorrect behavior.

For example, the output from a simulation code may look something similar to:

> -- Neighborlist stats:
6392 normal updates / 167 forced updates / 0 dangerous updates <br>
n_neigh_min: 16 / n_neigh_max: 100 / n_neigh_avg: 64.63 <br>
shortest rebuild period: 6<br>

where it is clear that we did not have any dangerous builds. This also provides information regarding the minimum time between neighborlist updates.

Often more detailed information can be acquired.  For example, 50% of the computational time is actually spent on the neighborlist.

> `Simulation:     3.1s | 100.0%  `<br>
  `      Integrate:     0.5s | 16.5% `<br>
  `              NVT step 1:     0.1s | 4.2% `<br>
  `              NVT step 2:     0.1s | 3.0% `<br>
  `              Net force:      0.1s | 2.9% `<br>
  `              Thermo:         0.1s | 3.7% `<br>
  `              Self:           0.1s | 2.8% `<br> 
  `      Neighbor:      1.6s | 50.0% `<br>
  `              Cell:           0.0s | 0.8% `<br>
  `                      compute:     0.0s | 0.5% `<br>
  `                      init:        0.0s | 0.2% `<br>
  `              compute:        1.5s | 48.2% `<br>
  `              dist-check:     0.0s | 1.0% `<br> 
  `      Pair lj:       0.9s | 30.2% `<br>
  `      SFCPack:       0.1s | 3.1% `<br>
  `      Self:          0.0s | 0.2% `<br>

### Exercises

1) Using the simple monoatomic LJ system from the Anatomy of a Script file, djust the nlist buffer to examine the performance as a function of increasing the skin? How small can you make the skin before dangerous builds are detected? How does this impact the speed of the code (e.g. TPS). 
 See: http://hoomd-blue.readthedocs.io/en/stable/module-md-nlist.html#hoomd.md.nlist.cell
 
2) What happens if you set the check period to 1? 10? 100? 1000? Can you speed up the simulation and still avoid dangerous builds?
